<a href="https://colab.research.google.com/github/vemuriPradyumna/Text-Summarization/blob/main/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

In [3]:
%cd /content

/content


In [4]:
from google.colab import drive
 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
df = pd.read_csv('gdrive/MyDrive/Colab/Text Summarization/tennis_articles_v4.csv')

In [7]:
df.head()

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [8]:
for i in range(0,len(df['article_id']),1):
    print(i+1,'\n\n',len(df['article_text'][i]),'\n\n',df['article_text'][i])

1 

 1561 

 Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're 

In [9]:
sentences = []
for s in df['article_text']:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [10]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [11]:
print(clean_sentences)

['maria sharapova has basically no friends as tennis players on the wta tour ', 'the russian player has no problems in openly speaking about it and in a recent interview she said   i don t really hide any feelings too much ', 'i think everyone knows this is my job here ', 'when i m on the courts or when i m on the court playing  i m a competitor and i want to beat every single person whether they re in the locker room or across the net so i m not the one to strike up a conversation about the weather and know that in the next few minutes i have to go and try to win a tennis match ', 'i m a pretty competitive girl ', 'i say my hellos  but i m not sending any players flowers as well ', 'uhm  i m not really friendly or close to many players ', 'i have not a lot of friends away from the courts  ', 'when she said she is not really close to a lot of players  is that something strategic that she is doing ', 'is it different on the men s tour than the women s tour ', ' no  not at all ', 'i thin

In [12]:
stop_words = stopwords.words('english')

In [13]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [14]:
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [15]:
print(clean_sentences)

['maria sharapova basically friends tennis players wta tour', 'russian player problems openly speaking recent interview said really hide feelings much', 'think everyone knows job', 'courts court playing competitor want beat every single person whether locker room across net one strike conversation weather know next minutes go try win tennis match', 'pretty competitive girl', 'say hellos sending players flowers well', 'uhm really friendly close many players', 'lot friends away courts', 'said really close lot players something strategic', 'different men tour women tour', '', 'think sport mean friends everyone categorized tennis player going get along tennis players', 'think every person different interests', 'friends completely different jobs interests met different parts life', 'think everyone thinks tennis players greatest friends', 'ultimately tennis small part', 'many things interested', 'basel switzerland ap roger federer advanced th swiss indoors final career beating seventh seeded

In [16]:
word_embeddings = {}
f = open('gdrive/MyDrive/Colab/Text Summarization/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [17]:
print(len(word_embeddings))

400000


In [18]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [19]:
print("Number of cleaned sentences : {} \n\nNumber of sentence vectors: {}".format(len(clean_sentences),len(sentence_vectors)))

Number of cleaned sentences : 119 

Number of sentence vectors: 119


In [20]:
print("Number of original sentences : {}".format(len(sentences)))
sim_mat = np.zeros([len(sentences), len(sentences)])

Number of original sentences : 119


In [21]:
print("Building Cosine Similarity")

Building Cosine Similarity


In [22]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [23]:
print(type(sim_mat)," size : ",len(sim_mat))

<class 'numpy.ndarray'>  size :  119


In [24]:
print(sim_mat)

[[0.         0.64378339 0.59156996 ... 0.72776717 0.5850122  0.68277591]
 [0.64378339 0.         0.83267683 ... 0.77598292 0.79842889 0.87404495]
 [0.59156996 0.83267683 0.         ... 0.73326033 0.75388014 0.8130123 ]
 ...
 [0.72776717 0.77598292 0.73326033 ... 0.         0.74423134 0.87150967]
 [0.5850122  0.79842889 0.75388014 ... 0.74423134 0.         0.78779906]
 [0.68277591 0.87404495 0.8130123  ... 0.87150967 0.78779906 0.        ]]


In [25]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [26]:
print(type(scores)," size : ",len(scores))

<class 'dict'>  size :  119


In [27]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [28]:
print(ranked_sentences)

[(0.009293791287246103, "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match."), (0.009260614586854338, 'Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.'), (0.00924452157157331, 'Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.'), (0.009213900328590948, '"I felt like the best weeks that I had to get to know players when I was playing were the Fed Cup weeks or the Olympic weeks, not necessarily during the tournaments.'), (0.0092035881645

In [29]:
title = input("Enter File Name : ")
txt = ""
for i in range(10):
    txt = txt + ranked_sentences[i][1] +  "\n\n"
    
print(txt)


Enter File Name : Tennis
When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.

Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.

Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.

"I felt like the best weeks that I had to get to know players when I was playing were the Fed Cup weeks or the Olympic weeks, not necessarily during the tournaments.

Currently in ninth place, Nishikori with a win could move to within 125 points of the cut for t

In [31]:
f = open("gdrive/MyDrive/Colab/Text Summarization/{}.txt".format(title),"w")
f.write(txt)
f.close()